# **Tutorial GROVER - DNA Language Model**

Melissa Sanabria, Jonas Hirsch, Pierre M. Joubert, Anna R. Poetsch

Biomedical Genomics, Biotechnology Center, Center for Molecular and Cellular Bioengineering, Technische Universität Dresden  
melissa.sanabria@tu-dresden.de arpoetsch@gmail.com


# Overview

This tutorial will show you how to use the DNA language model GROVER to perform fine-tuning tasks to investigate genome biology.

[GROVER](https://www.nature.com/articles/s42256-024-00872-0) ("Genome Rules Obtained Via Extracted Representations") is a foundation DNA language model with an optimized vocabulary for the human genome. It has been pre-trained on the human genome and needs to be trained a second time, or fine-tuned, to perform specific tasks.

For this tutorial we chose an end-to-end example to fine-tune the pre-tained GROVER to predict DNA binding by the CCCTC-Binding Factor (CTCF). The task is to recognize which sites that contain a CTCF binding motif are actually bound by the protein. We will be using ChIP-seq data from HepG2 cells obtained from ENCODE.  

Throughout this tutorial, you will find ***MODIFY*** signs that indicate pieces of code that are task specific, meaning that they are specific for CTCF binding site prediction, and you can modify them for your desired task.

**WARNING**: Even though Google Colab gives you access to a GPU, which is necessary for this computationally expensive work, the free version only gives you access to a very simple device and the training might take a while.

In the version of Colab, which is free of charge, notebooks can run for a maximum of 12 hours, depending on availability and your usage patterns.  

## Setup

First, save a copy of this notebook in your Google Drive by navigating to 'File' then 'Save a copy in Drive...'.

Once you've opened your own copy, make sure you have enabled the GPU runtime for Google Colab by navigating to the menu 'Runtime', select 'Change runtime type' and set the runtime to 'T4 GPU'.


## Install dependencies

In [1]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=edafcf0b509f79a7bd91c4316d3c4de557723e97b2efe6a2076f77bd037a7607
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


## Import necessary packages

In [2]:
import pandas as pd
import wget
import os
import numpy as np

from sklearn.metrics import matthews_corrcoef, accuracy_score, precision_score, recall_score, f1_score

import transformers
from transformers import AutoTokenizer, Trainer, AutoModelForSequenceClassification, TrainingArguments
from torch.utils.data import Dataset

# Prepare Data

**MODIFY!**

This section is task dependent.

We have listed each CTCF motif in the genome which has been detected by the FIMO software. For this specific task, we calculate the center of the motif and then add 500 nucleotides up- and downstream. This is to capture the sequence context of the motif, which is what GROVER was trained to work with.

# Create dataset

From now on, we come back to general instructions that apply for any fine-tuning task.

Please remember that to do this yourself you will need a data frame with at least two columns: label, sequence

In [3]:
CTCF_dataset_url = "https://datashare.tu-dresden.de/s/pjb5gcWGGrbcARN/download/CTCF_dataset.tsv"
file_name = "CTCF_dataset.tsv"
if os.path.exists(file_name):
  os.remove(file_name) # if exists, remove it directly

print("Starting downloading")
file_name = wget.download(CTCF_dataset_url, out=file_name)
print(file_name)

Starting downloading
CTCF_dataset.tsv


In [4]:
CTCF_dataset = pd.read_csv('CTCF_dataset.tsv', sep='\t')
CTCF_dataset

,chromosome,start_of_bin,end_of_bin,label,sequence
0,chr1,168067890,168068890,0,GTGAGCCTGGGGCTGCTCTGAGCTGAGCCTTGAGCCCAGGCCAGGA...
1,chr17,45355603,45356603,0,ATATTGACTCTCGCTCTCCAAGAAGAGTATGAAATAATAGTAGTTA...
2,chr19,11551317,11552317,1,AGTGGCATGATCTTTGCTTACCCCAACCTCCACATCCGGGTTCAAG...
3,chrX,38034101,38035101,1,TATGCACATTCTCTCTAAAATGTTGTAATACTGGCATTAATCTAAA...
4,chr4,28445075,28446075,0,TTATCCTAATGCATGAATTTTCATCTAATACATGAAAGGAGAAAAA...
...,...,...,...,...,...
85482,chrY,27637470,27638470,0,TGGCCTGACAGCCTCTACCAGTCCTGCTGTCCCTTGGCTGAGAAAC...
85483,chrY,28133554,28134554,0,ACCACAGGCATGCACCACCAAGCTCAGCTTATTTTTTTTGTATTTT...
85484,chrY,28168346,28169346,0,AGAAGAAGCAGAGTTCTATTTGGCATTAAAACGGTTCAAAGTGTTG...
85485,chrY,28427226,28428226,0,TAGTGCCTGGCATAAAGTAAGCACGTATTGAGTAAAGTAAACACTC...


After getting our dataset, we need to split the samples into train, validation and test. We will go for the standard partitions, 80% for training, 10% for testing and 10% for validation.

In [5]:
train = CTCF_dataset.sample(frac=0.8, random_state=0)
validation = CTCF_dataset.drop(train.index)
test = validation.sample(frac=0.5, random_state=0)
validation = validation.drop(test.index)

train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)
test = test.reset_index(drop=True)

# GROVER in action

So far, we have not mentioned any Language Model terminology. Now, we need to change our sequence from nucleotides to tokens (or words). For this, we will download the tokenizer available in the higgingface project for GROVER. For this, we can use the model name *PoetschLab/GROVER*

In [6]:
from transformers import PreTrainedTokenizerFast

In [7]:
tokenizer = AutoTokenizer.from_pretrained("PoetschLab/GROVER")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/77.0 [00:00<?, ?B/s]

Then, we need to download the pre-trained GROVER model.

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("PoetschLab/GROVER")

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/348M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at PoetschLab/GROVER and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In order to make your work easier, we create a class to process the Dataset created for Grover.


In [9]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, texts, labels, tokenizer):

        super(SupervisedDataset, self).__init__()

        sequences = [text for text in texts]

        output = tokenizer(
            sequences,
            add_special_tokens=True,
            max_length=310,
            padding="longest",
            return_tensors="pt",
            truncation=True
        )

        self.input_ids = output["input_ids"]
        self.attention_mask = output["attention_mask"]
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i):
        return dict(
            input_ids=self.input_ids[i],
            labels=self.labels[i],
            attention_mask=self.attention_mask[i]
        )

In [10]:
train_dataset = SupervisedDataset(train.sequence, train.label, tokenizer)
test_dataset = SupervisedDataset(test.sequence, test.label, tokenizer)
val_dataset = SupervisedDataset(validation.sequence, validation.label, tokenizer)

## Train model

For the training process we choose:

*   an Adam optimizer but you can explore the [optimizers](https://huggingface.co/transformers/v3.0.2/main_classes/optimizer_schedules.html) available in huggingface
*   learning rate of 0.000001 since for fine-tuning tasks it is better to have a very low learning rate
*   a total number of epochs of 4

A higher number of epochs is ideal but Colab limits the amount of time you can run some code. We advice you to try with your own GPU resources with at least 10 epochs.

In order to make your work easier, we create a method to compute five classification metrics. Here we will see some metrics such as accuracy, f1 score, precision and recall. You can explore the [sklearn metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) to choose other options.

In [11]:
def calculate_metric_with_sklearn(logits: np.ndarray, labels: np.ndarray):
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(
            labels, predictions, average="macro", zero_division=0
        ),
        "matthews_correlation": matthews_corrcoef(
            labels, predictions
        ),
        "precision": precision_score(
            labels, predictions, average="macro", zero_division=0
        ),
        "recall": recall_score(
            labels, predictions, average="macro", zero_division=0
        ),
    }

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):  # Unpack logits if it's a tuple
        logits = logits[0]
    return calculate_metric_with_sklearn(logits, labels)

In [12]:
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable WandB logging if not needed

train_args = TrainingArguments(
    seed=42,
    output_dir="./output",  # Specify an output directory
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    learning_rate=0.000001,
    num_train_epochs=2,
    logging_dir="./logs",  # Optional: specify a logging directory
    run_name="CTCF_binder_prediction",  # Optional: specify a unique run name
)

trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=train_args
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-12-e33f7e7708fc>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Matthews Correlation,Precision,Recall
1,0.585600,0.593278,0.691075,0.639484,0.304672,0.670549,0.636068
2,0.563800,0.582485,0.700082,0.658313,0.331711,0.679268,0.653447


TrainOutput(global_step=8550, training_loss=0.5890855606257567, metrics={'train_runtime': 7464.0483, 'train_samples_per_second': 18.325, 'train_steps_per_second': 1.145, 'total_flos': 2.1789809271804e+16, 'train_loss': 0.5890855606257567, 'epoch': 2.0})

## Test model

After training the model, we can see the performance of our model on the test set, which are samples that the model has not previously seen.

In [13]:
results = trainer.evaluate(eval_dataset=test_dataset)

In [14]:
results

{'eval_loss': 0.5692647695541382,
 'eval_accuracy': 0.7103416003743566,
 'eval_f1': 0.6631319540314187,
 'eval_matthews_correlation': 0.34473823751751864,
 'eval_precision': 0.6890342703906551,
 'eval_recall': 0.6571731572284262,
 'eval_runtime': 132.8079,
 'eval_samples_per_second': 64.364,
 'eval_steps_per_second': 8.049,
 'epoch': 2.0}

# Now, it's your turn

We use CTCF binding site as example but you can also use it for many other tasks such as promoter prediction, structural variants, integration sites of transposable elements, other binding sites, and many more.